<a href="https://colab.research.google.com/github/satani99/generative_deep_learning/blob/main/03_variational_autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch.nn as nn 
import torch

In [5]:
# The Encoder
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()

    self.conv0 = nn.Conv2d(3, 32, kernel_size=3, stride=2)
    self.leaky_relu1 = nn.LeakyReLU(32)
    self.conv1 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
    self.leaky_relu2 = nn.LeakyReLU(64)
    self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
    self.leaky_relu3 = nn.LeakyReLU(64)
    self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
    self.leaky_relu4 = nn.LeakyReLU(64)
    self.linear = nn.Linear(1024, 2)

  def forward(self, x):
    x = self.conv0(x)
    x = self.leaky_relu1(x)
    x = self.conv1(x)
    x = self.leaky_relu2(x)
    x = self.conv2(x)
    x = self.leaky_relu3(x)
    x = self.conv3(x)
    x = self.leaky_relu4(x)
    x = self.flatten(x, 1)
    output = self.linear(x)
    return output
              

In [6]:
encoder = Encoder()
print(encoder)

Encoder(
  (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
  (leaky_relu1): LeakyReLU(negative_slope=32)
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (leaky_relu2): LeakyReLU(negative_slope=64)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (leaky_relu3): LeakyReLU(negative_slope=64)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (leaky_relu4): LeakyReLU(negative_slope=64)
  (linear): Linear(in_features=1024, out_features=2, bias=True)
)


In [11]:
class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()

    self.linear = nn.Linear(2, 3136)
    self.conv_t_0 = nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1)
    self.leaky_relu5 = nn.LeakyReLU(64)
    self.conv_t_1 = nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1)
    self.leaky_relu6 = nn.LeakyReLU(64)
    self.conv_t_2 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2)
    self.leaky_relu7 = nn.LeakyReLU(32)
    self.conv_t_3 = nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2)

  def forward(self, x):
    x = self.linear(x)
    x = self.leaky_relu5(self.conv_t_0(x.view(-1, 7, 7, 64)))
    x = self.leaky_relu6(self.conv_t_1(x))
    x = self.leaky_relu7(self.conv_t_2(x))
    output = torch.sigmoid(x)
    return output
    

In [12]:
decoder = Decoder()
print(decoder)

Decoder(
  (linear): Linear(in_features=2, out_features=3136, bias=True)
  (conv_t_0): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (leaky_relu5): LeakyReLU(negative_slope=64)
  (conv_t_1): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (leaky_relu6): LeakyReLU(negative_slope=64)
  (conv_t_2): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2))
  (leaky_relu7): LeakyReLU(negative_slope=32)
  (conv_t_3): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2))
)
